## The Cart Pole


This problem requires balancing a pole up on a cart for as long as possible.

### State: 

(0) Cart Position (-2.4 to 2.4)

(1) Cart Velocity (-infinity to infinity)

(2) Pole Angle (-41.8 to 41.8)

(3) Pole Velocity at Tip (-infinity to infinity)

### Actions: 

Left or Right (Force of -1 or +1 to the cart)

### Reward: 

For each second the pole is upright, reward + 1

Max reward: 200 (+1 for a total of 200 time steps)

### Starting State:

All state variables assigned a uniform random value, magnitude less than 0.05

### Termination:

The episode ends when:

(1) The pole is more than 12 degrees from vertical

(2) The cart moves more than 2.4 units from the center

(3) Episode length greater than 200

# Install Pre-requisites/Dependencies

The following dependencies are required to get the gym environment working and also to display the gym environment as a video

In [1]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym > /dev/null 2>&1

     |████████████████████████████████| 788kB 6.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 50.3.2
    Uninstalling setuptools-50.3.2:
      Successfully uninstalled setuptools-50.3.2


# Configure Colab to display video of our agent!

The code below simply configures Colab to display a video of the environment for us to visualize.

In [3]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import io
import base64
import numpy as np
from IPython.display import HTML
from IPython import display as ipythondisplay

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [4]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Environment Visualization

Gym is a very useful package by OpenAI (https://gym.openai.com/) that comes built in with many environments to train our agents on.

Here, we will use the CartPole environment using:

*env = gym.make('CartPole-v0')*

We reset the environment to its initial state using:

*env.reset()*

We visualize the environment using:

*env.render()*

We take an action in the environment using:

*env.step(action)*

We create a function "visualize" that lets us visualize the actions of an agent using decision_function to choose the actions.

In [10]:
 env = wrap_env(gym.make('CartPole-v0'))
 
 def visualize(decision_function):
  env = wrap_env(gym.make('CartPole-v0'))
  state = env.reset()
  totalreward = 0
  for t in range(200):
      env.render()
      action = decision_function(state)
      state, reward, done, _ = env.step(action)
      totalreward += reward
      if done:
          break
  print("Episode finished after {} timesteps with reward {}".format(t+1, totalreward))
  env.close()
  show_video()

We visualize the actions of a random agent:

In [11]:
def random_action(state):
  return env.action_space.sample()

In [12]:
visualize(random_action)

Episode finished after 22 timesteps with reward 22.0


# Check out the actions

In [13]:
print('Number of actions:', env.action_space.n)

Number of actions: 2


In [19]:
# Run this cell many times and see what is output
print('A random action is:', env.action_space.sample())

A random action is: 1


# Check out the observations

In [20]:
env.reset()
state, reward, done, info = env.step(0)
print('State:', state)
print('Reward:', reward)
print('Done:', done)
print('Info:', info)

State: [-0.01258419 -0.17359022 -0.01007264  0.24467366]
Reward: 1.0
Done: False
Info: {}


# Agent 1: Human-expertise guided approach (Heuristics)

One way to solve cart pole is to just use simple heuristics (some external knowledge about what is good and bad) to determine the action.

One way is to say that when the pole angle is negative (pole tilted to the left), then we move the cart to the left (action 0) so as to try to regain balance.

Otherwise, we move it to the right (action 1)

In [21]:
def select_action_good(state):
    cart_pos, cart_vel, pole_ang, pole_vel = state
    if pole_ang < 0:
        return 0
    else:
        return 1

In [24]:
visualize(select_action_good)

Episode finished after 36 timesteps with reward 36.0


# Can we do better by taking into account other factors like the pole velocity?

### Individual exercise / discussion

Try to configure the code below using if else statements, and try to get the pole to consistently reach **timestep 200** if possible!

Note:

- Action value of 0 represents moving the cart to the left
- Action value of 1 represents moving the cart to the right
- Negative angles/position/velocity represent towards the left
- Positive angles/position/velocity represent towards the right

In [25]:
def select_action_better(state):
    cart_pos, cart_vel, pole_ang, pole_vel = state
    if pole_vel < 0:
        return 0
    elif pole_ang < 0:
        return 0
#     elif cart_pos < 0:
#         return 0
#     elif cart_vel < 0:
#         return 0
    else:
        return 1

In [28]:
visualize(select_action_better)

Episode finished after 131 timesteps with reward 131.0


# Agent 2: Doing it without human expertise - Deep Q-Network (DQN) [Advanced]

The following code was modified from: https://github.com/gsurma/cartpole/blob/master/cartpole.py

We simply train a single neural network that takes in the state space as input, and outputs two values:

- Q-value for Action 0
- Q-value for Action 1

The chosen action will then be whichever action gives the highest Q-value. 

You can treat the Q-value as the proxy for the total accumulated future reward that you can get from a state if you choose an action.

# Additional Concepts (Optional)

### Compute Q-learning Temporal Difference error:

(This is equivalent to the difference between what the expected reward of what the current step predicts, compared to what the next step predicts

-> Think of it as the further you go into time, the more accurate your prediction of the outcome because there is less uncertainty 

-> Hence if we try to match current step prediction with the next step prediction's values, we will be more and more accurate at predicting the future)


The loss $L$ which we use to train the neural network is defined as:

$$ L = { 1 \over N} \sum_i [ Q_{\theta}(s,a) - Q_{reference}(s,a) ] ^2 $$

with $Q_{reference}$ defined as:

$$ Q_{reference}(s,a) = r(s,a) + \gamma \cdot max_{a'} Q_{target}(s', a') $$

Where
* $Q_{target}(s',a')$ denotes q-value of next state and next action predicted by __target_network__
* $Q_{\theta}(s,a)$ denotes q-value of current state and action predicted by __target_network__
* $s, a, r, s'$ are current state, action, reward and next state respectively
* $\gamma$ is a discount factor to discount future rewards
* $N$ is the number of experiences sampled from the replay buffer

### Replay Buffer 
In order to prevent high correlation between the input samples, we store our accumulated experience into a replay buffer, and then sample randomly from it to learn. 

A human analog is that we replay our experiences in the day in our dreams, and learn from our dreams at night.

# The Code

Since we use a neural network, we import tensorflow/keras (https://keras.io/) for the neural network training.

We also import random (https://docs.python.org/3/library/random.html) so that we can randomly sample our replay buffer.

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

import random

We set certain hyperparameters (= not trained within the model) in order to influence the way the agent learns.

When you are more familiar with deep learning/reinforcement learning, you can adjust this, but for now, this has been tuned and you should just use this.

In [30]:
## HYPERPARAMETERS
# These can be modified for you to understand the importance of these values
GAMMA = 0.99 # Discounted value of future rewards: Between 0 and 1
LEARNING_RATE = 0.001 # How fast you want the neural network to learn: Between 0 and 1
MEMORY_SIZE = 1000 # Number of memories the replay buffer can store: Positive Integer
BATCH_SIZE = 16 # Number of samples to learn from replay buffer: Positive Integer, lower than MEMORY_SIZE
EPSILON = 0.95 # Decay factor for exploration: Between 0 and 1

DQNSolver is a class that uses DQN in order to solve the Cart Pole problem

It has:
- an __init__ function, which initializes the solver
- a remember function, in order to store memory into the replay buffer
- an act function, which selects an action randomly a fraction **exploration_rate** of the time, and otherwise selects the best action (greedy selection)
- an experience_replay function, which samples memories to learn from the replay buffer

In [31]:
class DQNSolver:
    def __init__(self, observation_space, action_space):
        self.action_space = action_space
        self.memory = []
        self.exploration_rate = 1
        self.model = Sequential()
        self.model.add(Input(shape = (observation_space,))) # Input layer with observation_space number of nodes to represent state space
        self.model.add(Dense(self.action_space, activation="linear"))  # Output layer: Must have self.action_space number of nodes
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        # add a memory to the replay buffer
        self.memory.append((state, action, reward, next_state, done))
        # only keep the most recent MEMORY_SIZE number of memories
        self.memory =  self.memory[-MEMORY_SIZE:]

    def act(self, state, greedy = False):
        # Only explore if less than exploration rate
        if np.random.rand() < self.exploration_rate and not greedy:
            return env.action_space.sample()
        # Otherwise, choose the best action
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def experience_replay(self):
        # only start sampling the replay buffer when there is enough memory
        if len(self.memory) < BATCH_SIZE:
            return
        # sample BATCH_SIZE amount of memory from the memory buffer
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, done in batch:
            # if terminal step, then take the terminal reward
            if done:
                q_update = reward
            # if not terminal step, then take this step's reward plus future accumulated rewards
            else:
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
                
            # use the neural network to predict the q_values for the current state
            q_values = self.model.predict(state)
            # update the q_value of the action taken with the future predicted value q_update
            q_values[0][action] = q_update
            # train the neural network with the updated q_values
            self.model.fit(state, q_values, verbose=0)
            
        # Explore less next run by decaying exploration rate
        self.exploration_rate *= EPSILON

Now we run the environment and let the DQN algorithm learn:

If the agent does not perform well, run the code again.

This is because the rate of learning in DQN is highly dependent on the starting configuration, so a good starting state will enable it to learn much better.

In [34]:
env = wrap_env(gym.make('CartPole-v0'))
observation_space = env.observation_space.shape[0]
action_space = env.action_space.n
dqn_solver = DQNSolver(observation_space, action_space)
for run in range(20):
    state = env.reset()
    state = np.reshape(state, [1, observation_space])
    # give some time for buffer to fill up
    if run > 3:
        dqn_solver.experience_replay()
    for step in range(200):
        action = dqn_solver.act(state)
        state_next, reward, done, info = env.step(action)
        state_next = state_next.reshape([1, observation_space])
        dqn_solver.remember(state, action, reward, state_next, done)
        state = state_next
        if done:
            print ("Run: " + str(run+1) + ", exploration_rate: " + str(dqn_solver.exploration_rate) + ", score: " + str(step))
            break

Run: 1, exploration_rate: 1, score: 22
Run: 2, exploration_rate: 1, score: 24
Run: 3, exploration_rate: 1, score: 23
Run: 4, exploration_rate: 1, score: 29
Run: 5, exploration_rate: 0.95, score: 21
Run: 6, exploration_rate: 0.9025, score: 21
Run: 7, exploration_rate: 0.8573749999999999, score: 27
Run: 8, exploration_rate: 0.8145062499999999, score: 21
Run: 9, exploration_rate: 0.7737809374999999, score: 49
Run: 10, exploration_rate: 0.7350918906249998, score: 9
Run: 11, exploration_rate: 0.6983372960937497, score: 66
Run: 12, exploration_rate: 0.6634204312890623, score: 14
Run: 13, exploration_rate: 0.6302494097246091, score: 61
Run: 14, exploration_rate: 0.5987369392383786, score: 66
Run: 15, exploration_rate: 0.5688000922764596, score: 17
Run: 16, exploration_rate: 0.5403600876626365, score: 44
Run: 17, exploration_rate: 0.5133420832795047, score: 44
Run: 18, exploration_rate: 0.48767497911552943, score: 50
Run: 19, exploration_rate: 0.46329123015975293, score: 23
Run: 20, exploratio

# Visualize your DQN agent

In [35]:
def dqn_agent(state):
  return dqn_solver.act(state.reshape([1, observation_space]), greedy = True)

In [36]:
visualize(dqn_agent)

Episode finished after 105 timesteps with reward 105.0


# Concluding Remarks

One good thing about using neural networks is that you can easily train an agent with very complex / high-dimensional state spaces as inputs.
However, you lose out on the interpretability of the model.
Compare your DQN agent to that of the heuristic-based agent earlier using the following guiding questions:
- Which is easier to understand by humans?
- Which requires more human input?
- Which requires longer code?
- If someone were to ask you how your AI system works, which kind of model would you feel more confident explaining?